# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.121553e+02     1.414435e+00
 * time: 0.7087199687957764
     1     1.008747e+01     7.580316e-01
 * time: 2.112499952316284
     2    -1.187804e+01     9.568096e-01
 * time: 2.8052430152893066
     3    -3.429310e+01     6.947166e-01
 * time: 3.8299400806427
     4    -4.764192e+01     4.861383e-01
 * time: 4.883531093597412
     5    -5.702864e+01     1.910113e-01
 * time: 5.902583122253418
     6    -5.986840e+01     1.398062e-01
 * time: 6.594900131225586
     7    -6.094135e+01     4.970255e-02
 * time: 7.31464695930481
     8    -6.136843e+01     5.033988e-02
 * time: 8.001984119415283
     9    -6.162257e+01     4.667663e-02
 * time: 8.69707202911377
    10    -6.183670e+01     2.488093e-02
 * time: 9.432886123657227
    11    -6.200095e+01     2.034833e-02
 * time: 10.127192974090576
    12    -6.208145e+01     1.960965e-02
 * time: 10.785660982131958
    13    -6.214116e+01     1.318303e-02
 * time: 11.464699983596802
    14

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671062
    AtomicLocal         -18.8557696
    AtomicNonlocal      14.8522669
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485371 
    Xc                  -19.3336819

    total               -62.261666461284
